In [2]:
import pandas as pd
import numpy as np
from implicit.datasets.lastfm import get_lastfm

In [3]:
artists, users, artist_user_plays = get_lastfm()

0.00B [00:00, ?B/s]

In [6]:
from implicit.nearest_neighbours import bm25_weight

# weight the matrix, both to reduce impact of users that have played the same artist thousands of times
# and to reduce the weight given to popular items
artist_user_plays = bm25_weight(artist_user_plays, K1=100, B=0.8)

# get the transpose since the most of the functions in implicit expect (user, item) sparse matrices instead of (item, user)
user_plays = artist_user_plays.T.tocsr()

In [7]:
from implicit.als import AlternatingLeastSquares

model = AlternatingLeastSquares(factors=64, regularization=0.05, alpha=2.0)
model.fit(user_plays)

/Users/m0saan/miniforge3/envs/fastai/lib/python3.11/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 10 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/15 [00:00<?, ?it/s]

In [8]:
userid = 12345
ids, scores = model.recommend(userid, user_plays[userid], N=10, filter_already_liked_items=False)

In [9]:
ids, scores

(array([216218, 166155, 192493, 254178, 272322,  79684,  30373, 157639,
        163659, 277688], dtype=int32),
 array([1.0420482 , 1.0190771 , 0.99050224, 0.986747  , 0.9820544 ,
        0.9725242 , 0.97222173, 0.9641731 , 0.94615906, 0.9439356 ],
       dtype=float32))

In [10]:
# Use pandas to display the output in a table, pandas isn't a dependency of implicit otherwise
import numpy as np
import pandas as pd
pd.DataFrame({"artist": artists[ids], "score": scores, "already_liked": np.in1d(ids, user_plays[userid].indices)})

,artist,score,already_liked
0,puissance,1.042048,True
1,laibach,1.019077,False
2,mortiis,0.990502,True
3,the coffinshakers,0.986747,True
4,type o negative,0.982054,True
5,d-a-d,0.972524,True
6,arditi,0.972222,True
7,karjalan sissit,0.964173,False
8,kreuzweg ost,0.946159,True
9,von thronstahl,0.943936,True


In [11]:
# get related items for the beatles (itemid = 25512)
ids, scores= model.similar_items(252512)

# display the results using pandas for nicer formatting
pd.DataFrame({"artist": artists[ids], "score": scores})

,artist,score
0,the beatles,1.000000
1,the beach boys,0.992893
2,the rolling stones,0.992336
3,john lennon,0.991617
4,bob dylan,0.991472
5,the who,0.991404
6,david bowie,0.991223
7,simon & garfunkel,0.990994
8,led zeppelin,0.990192
9,pink floyd,0.989226


In [13]:
from implicit.evaluation import precision_at_k as p_at_k
p_at_k(model, user_plays, user_items=None, K=10, num_threads=0, check_intersections=True)

TypeError: precision_at_k() takes at least 3 positional arguments (2 given)

In [ ]:
train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10).mean()

train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))